In [3]:
# Import necessary libraries
import pandas as pd
from gurobipy import Model, GRB

# Load datasets
farms = pd.read_csv('/Users/Sam/Downloads/farms.csv')
processing = pd.read_csv('/Users/Sam/Downloads/processing.csv')
centers = pd.read_csv('/Users/Sam/Downloads/centers.csv')

# Define dimensions
num_farms = len(farms)
num_plants = len(processing)
num_centers = len(centers)

# Extract transportation cost data
transport_cost_farm_to_plant = [[farms.iloc[i, 4+j] for j in range(num_plants)] for i in range(num_farms)]
transport_cost_plant_to_center = [[processing.iloc[j, 4+k] for k in range(num_centers)] for j in range(num_plants)]

# Function to initialize a new model
def initialize_model():
    # Create a new model
    model = Model('BioAgri Optimization')

    # Create decision variables
    x = model.addVars(num_farms, num_plants, vtype=GRB.CONTINUOUS, name="x")  # Raw material transport
    y = model.addVars(num_plants, num_centers, vtype=GRB.CONTINUOUS, name="y")  # Fertilizer transport

    # Objective function: Minimize total cost
    model.setObjective(
        sum(x[i, j] * (farms.loc[i, 'Cost_Per_Ton'] + transport_cost_farm_to_plant[i][j])
            for i in range(num_farms) for j in range(num_plants)) +
        sum(y[j, k] * (processing.loc[j, 'Processing_Cost_Per_Ton'] + transport_cost_plant_to_center[j][k])
            for j in range(num_plants) for k in range(num_centers)),
        GRB.MINIMIZE
    )

    return model, x, y


In [4]:
# Initialize a new model
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Optimize the model
model.optimize()

# Output results for (a) and (b)
if model.status == GRB.OPTIMAL:
    print("Optimal Cost:", model.objVal)
else:
    print("No optimal solution found.")


Set parameter Username
Set parameter LicenseID to value 2609998
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 387 rows, 6318 columns and 17118 nonzeros
Model fingerprint: 0x3aa78cc0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve time: 0.00s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   2.902000e+04   0.000000e+00      0s
     330    2.2970900e+06   0.000000e+00   0.000000e+00      0s

Solved in 330 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.297089973e+06
Optimal Cost: 2297089.972722625


* c

In [5]:
# Initialize a new model for part (c)
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add regional restriction constraints
for j in range(num_plants):
    plant_region = processing.loc[j, 'Region']
    for k in range(num_centers):
        center_region = centers.loc[k, 'Region']
        if plant_region != center_region:
            model.addConstr(y[j, k] == 0, name=f"RegionalRestriction_Plant{j}_Center{k}")

# Optimize the model
model.optimize()

# Output results for (c)
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Regional Restriction):", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1761 rows, 6318 columns and 18492 nonzeros
Model fingerprint: 0x9c296277
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 1374 rows and 1374 columns
Presolve time: 0.00s
Presolved: 387 rows, 4944 columns, 14370 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2479485e+05   3.627500e+03   0.000000e+00      0s
     320    2.3234885e+06   0.000000e+00   0.000000e+00      0s

Solved in 320 iterations and 0.01 seconds (0.01 work units)
Optimal objective  2.323488497e+06
Optimal Cost (Regional Restriction): 2323488.4970226064


* d

In [7]:
# Initialize a new model for part (d)
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add quality restriction constraints
for i in range(num_farms):
    if farms.loc[i, 'Quality'] < 3:
        for j in range(num_plants):
            model.addConstr(x[i, j] == 0, name=f"QualityRestriction_Farm{i}_Plant{j}")

# Optimize the model
model.optimize()

# Output results for (d)
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Highest Quality Only):", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3519 rows, 6318 columns and 20250 nonzeros
Model fingerprint: 0xa45bebf3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 3306 rows and 3132 columns
Presolve time: 0.00s
Presolved: 213 rows, 3186 columns, 7722 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   3.627500e+03   0.000000e+00      0s
     253    5.7129767e+06   0.000000e+00   0.000000e+00      0s

Solved in 253 iterations and 0.01 seconds (0.01 work units)
Optimal objective  5.712976673e+06
Optimal Cost (Highest Quality Only): 5712976.672909363


* e

In [8]:
# Initialize a new model for part (e) - Sourcing Risk Mitigation
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add sourcing risk mitigation constraint
total_supply = farms['Bio_Material_Capacity_Tons'].sum()
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= 0.03 * total_supply, 
                    name=f"SourcingRisk_Plant{j}")

# Optimize the model
model.optimize()

# Output results for (e) Part 1
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Sourcing Risk Mitigation):", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 405 rows, 6318 columns and 21600 nonzeros
Model fingerprint: 0x52a9cb0c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.01s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   3.627500e+03   0.000000e+00      0s
     408    2.3117826e+06   0.000000e+00   0.000000e+00      0s

Solved in 408 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.311782560e+06
Optimal Cost (Sourcing Risk Mitigation): 2311782.559847675


In [9]:
# Initialize a new model for part (e) - Supply Risk Mitigation
model, x, y = initialize_model()

# Add basic constraints
for i in range(num_farms):
    model.addConstr(sum(x[i, j] for j in range(num_plants)) <= farms.loc[i, 'Bio_Material_Capacity_Tons'], 
                    name=f"FarmSupply_{i}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) <= processing.loc[j, 'Capacity_Tons'], 
                    name=f"PlantCapacity_{j}")
for k in range(num_centers):
    model.addConstr(sum(y[j, k] for j in range(num_plants)) == centers.loc[k, 'Requested_Demand_Tons'], 
                    name=f"CenterDemand_{k}")
for j in range(num_plants):
    model.addConstr(sum(x[i, j] for i in range(num_farms)) == sum(y[j, k] for k in range(num_centers)), 
                    name=f"FlowBalance_{j}")

# Add supply risk mitigation constraint
for j in range(num_plants):
    for k in range(num_centers):
        model.addConstr(y[j, k] <= 0.5 * centers.loc[k, 'Requested_Demand_Tons'], 
                        name=f"SupplyRisk_Plant{j}_Center{k}")

# Optimize the model
model.optimize()

# Output results for (e) Part 2
if model.status == GRB.OPTIMAL:
    print("Optimal Cost (Supply Risk Mitigation):", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2223 rows, 6318 columns and 18954 nonzeros
Model fingerprint: 0x3179cd01
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 3e+04]
Presolve removed 1836 rows and 0 columns
Presolve time: 0.00s
Presolved: 387 rows, 6318 columns, 17118 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   5.441250e+03   0.000000e+00      0s
     398    2.3019498e+06   0.000000e+00   0.000000e+00      0s

Solved in 398 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.301949785e+06
Optimal Cost (Supply Risk Mitigation): 2301949.784839987
